In [1]:
from modelscope import snapshot_download
model_dir = snapshot_download("maple77/zpoint_large_embedding_zh")

2024-06-24 18:52:59,429 - modelscope - INFO - PyTorch version 2.3.0+cpu Found.
2024-06-24 18:52:59,431 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-06-24 18:52:59,432 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-06-24 18:52:59,485 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 eccbdad6389abeac8790b84f257211fb and a total number of 980 components indexed
Downloading: 100%|██████████| 172/172 [00:00<00:00, 327B/s]
Downloading: 100%|██████████| 51.0/51.0 [00:00<00:00, 123B/s]
Downloading: 100%|██████████| 341/341 [00:00<00:00, 877B/s]
Downloading: 100%|██████████| 7.01M/7.01M [00:00<00:00, 14.1MB/s]
Downloading: 100%|██████████| 1.21G/1.21G [00:04<00:00, 285MB/s] 
Downloading: 100%|██████████| 26.8k/26.8k [00:00<00:00, 65.5kB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 113B/s]
Downloading: 100%|████

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = PyPDFLoader("../soybean_konw.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents[:])
for idx, text in enumerate(texts):
    text.metadata["id"] = idx

ValueError: File path ../soybean_konw.pdf is not a valid file or url

In [2]:
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

top_k = 8

model_name = '/mnt/workspace/.cache/modelscope/hub/maple77/zpoint_large_embedding_zh'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vectorstore = Chroma.from_documents(documents=texts[:], embedding=hf)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": top_k}
)

/usr/local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [10]:
import pandas as pd

df = pd.read_excel('../learn_rag/soybean_question1.xlsx', sheet_name='Sheet1')

# 打印DataFrame的内容
column_lists = {col: df[col].tolist() for col in df.columns}
print(column_lists.keys())

dict_keys(['id', 'question', 'context'])


In [18]:
retriever_result = retriever.batch(column_lists['question'])

In [124]:
from retiever_eval_list import get_ht_score, get_mmr_score, get_retriever_res_list
retiever_evaldict = {}
for i in range(8):
    print('top_', i+1)
    res_retriever_list = get_retriever_res_list(retriever_result, i+1)
    ht_score = get_ht_score(column_lists['id'], res_retriever_list)
    mmr_score = get_mmr_score(column_lists['id'], res_retriever_list)
    # print('ht:', round(ht_score, 3))
    # print('mmr_score:', round(mmr_score, 3))
    soft__ht_score = get_soft_ht_score(column_lists['id'], res_retriever_list)
    soft__mmr_score = get_soft_mmr_score(column_lists['id'], res_retriever_list)
    # print('soft__ht_score:', round(soft__ht_score, 3))
    # print('soft__mmr_score:', round(soft__mmr_score, 3))
    res_retriever_list = get_retriever_res_list(retriever_result, i+1)
    ndcg_score = get_ndcg_score(column_lists['id'], res_retriever_list)
    # print('ndcg:', round(ndcg_score, 3))
    retiever_evaldict['top_' + str(i+1)] = {
        "ht_score": round(ht_score, 3),
        'mmr_score': round(mmr_score, 3),
        'soft_ht_score': round(soft__ht_score, 3),
        'soft_mmr_score': round(soft__mmr_score, 3),
        'ndcg': round(ndcg_score, 3),
    }

top_ 1
top_ 2
top_ 3
top_ 4
top_ 5
top_ 6
top_ 7
top_ 8


In [ ]:
retiever_evaldict

In [ ]:
def soft_ht_score(lab_index, retriever_res, top_k = 8, so = 3)->float:
    list1 = range(lab_index-so, lab_index+so)
    if any(element in retriever_res[:top_k] for element in list1):
        return 1.0
    else:
        return 0.0

def soft_mmr_score(lab_index, retriever_res, top_k = 8, so = 3)->float:
    list1 = range(lab_index-so, lab_index+so)
    ank = 0
    for element in list1:
        if element in retriever_res[:top_k]:
            ank += 1.0 / (retriever_res.index(element) + 1)
    return ank

def get_soft_ht_score(lab_index_list, retriever_list):
    ans = 0
    for index, num in enumerate(lab_index_list):
        ans += soft_ht_score(num, retriever_list[index])
    return ans / len(lab_index_list)

def get_soft_mmr_score(lab_index_list, retriever_list):
    ans = 0
    for index, num in enumerate(lab_index_list):
        ans += soft_mmr_score(num, retriever_list[index])
    return ans / len(lab_index_list)
for i in range(8):
    print('soft_top_', i+1)
    res_retriever_list = get_retriever_res_list(retriever_result, i+1)
    ht_score = get_soft_ht_score(column_lists['id'], res_retriever_list)
    mmr_score = get_soft_mmr_score(column_lists['id'], res_retriever_list)
    print('ht:', round(ht_score, 3))
    print('mmr_score:', round(mmr_score, 3))

In [ ]:
# id 5
#   2 3 4 5 6 7 8
#   0 1 2 3 2 1 0
# res [14, 2 , 3, 5, 16, 9, 8 , 9]
import math
def calucate_dcg(res_score):
    dcg = 0
    for i in range(len(res_score)):
        # print(math.log2(i+2))
        dcg += (res_score[i] / math.log2(i+2))
    return dcg

def calucate_idcg(res_score):
    idcg = 0
    sorted_res_score = sorted(res_score, reverse=True)
    for i in range(len(sorted_res_score)):
        # print(math.log2(i+2))
        idcg += (sorted_res_score[i] / math.log2(i+2))
    return idcg


def ndcg_score_cal(lab_index, res, top_k =8, so = 3):
    list1 = range(lab_index-so, lab_index+so+1)
    n = so + 1
    lab_score = list(range(n)) + list(range(n-2, -1, -1)) if n > 1 else []
    res_score = [0] * len(res)
    for index, ret_res in enumerate(res):
        if ret_res in list1:
            res_score[index] = lab_score[list1.index(ret_res)]
    dcg = calucate_dcg(res_score)
    idcg = calucate_idcg(res_score)
    if idcg == 0.0:
        return 0.0
    else:
        return dcg / idcg
    
    print(res_score)
    
def get_ndcg_score(lab_index_list, retriever_list):
    ans = 0
    for index, num in enumerate(lab_index_list):
        # print(num, retriever_list[index])
        ans += ndcg_score_cal(num, retriever_list[index])
    return ans / len(lab_index_list)

# res_retriever_list = get_retriever_res_list(retriever_result, 2)
# ndcg_score = get_ndcg_score(column_lists['id'], res_retriever_list)
# print('ndcg:', round(ndcg_score, 3))
for i in range(8):
    print('soft_top_', i+1)
    res_retriever_list = get_retriever_res_list(retriever_result, i+1)
    ndcg_score = get_ndcg_score(column_lists['id'], res_retriever_list)
    print('ndcg:', round(ndcg_score, 3))

In [117]:
column_lists['retriever_result'] = res_retriever_list

In [120]:
filename = "retriever_result1.json"
import json
with open(filename, 'w') as f:
    json.dump(column_lists, f, ensure_ascii=False, indent=4)

In [126]:
filename = "retiever_eval1.json"
import json
with open(filename, 'w') as f:
    json.dump(retiever_evaldict, f, ensure_ascii=False, indent=4)